# 2DConvolution in OpenCL
We can convert the C++ description of 2D convolution into OpenCL in a realtive straightforward manner. This requires splitting the code into two files -- a host file, and a kernel file. While this separation may seem forced, it naturally fits the **offload** model where the compute intensive portions of your application are moved to the accelerator. In our case, the nested for loops must be moved to the accelerator.

Writing kernel code in OpenCL requires a different way of thinking about computation. The kernel file for 2D convolution captures the operations that must be performed at a single pixel. This idea is important -- by only specifying the per-pixel computation in the kernel, we are relying on the OpenCL compiler and runtime to transform our code automatically to parallelize most effectively on the accelerator. This is also the reason behind OpenCL's platform portability claim. 

In [ ]:
__kernel void convolve2D(
    __global float *in,     // W*H input images
    __constant float *filt, // K*K filter kernel
    __global float *out,    // W*H output images
    int K,                  // filter resolution
    float pBias)            // constant offset/bias
{
    // get pixel position
    int W = get_global_size(0);
    int H = get_global_size(1);

    // get image resolution
    int x = get_global_id(0); 
    int y = get_global_id(1);

    float sum = 0;
    int c = 0;

    // loop over kernel rows
    for (int r = 0, r < K, r++) {
        // loop over kernel columns
        for(c = 0, c < K, c++) {
            sum += filt[r*K+c]*in[((y+r)*W+x)+c];
        }
    }
    out[y*W+x] = sum + pBias;
}

In the **convolve2D** OpenCL kernel code, we observe a few new portions. 
1. The **get_global_size** method is used to decide the image resolution $W$x$H$. 
2. The **get_global_id** is crucial to identify the $x$,$y$ position of the current pixel being processed. The concept of global ids is important. The OpenCL runtime will take this kernel and parallelize it across the global workgroup. Each workitem in the workgroup will have a unique global id. For this application, it represents the pixel position $x$,$y$. 
3. The nested for loops over the rows and columns of the image are gone! Only the loops over the kernels are left. This is tied to the idea of global workgroup size. When we set the global workgroup size to $W$x$H$, the OpenCL compiler automatically implicitly introduces the nested for loops over the image rows and columns for you. Additionally, the OpenCL runtime also parallelizes each pixel operation across the OpenCL device for you automatically. 

The workgroup is set in the host code which we discuss below.

The host code in OpenCL often must be written to target a given platform. It looks ugly in C/C++ but its mostly structurally same across all applications. We can break it down into the key building blocks listed below. With some differences, most applications will use this model.

First, we must include the correct OpenCL headers

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <CL/cl.h>
#include <CL/cl_ext.h>

Next, we must setup the OpenCL platform. Look, this following piece of code is not only ugly, its tedious, and annoying. There are C++ and PyOpenCL flows that significantly clean this up, but for our FPGA backend, we still need to understand this process. Bear with us. The structure is pretty broilerplate and with few modifications, you can use it for other applications. There is also extensive error checking, a good practice, to help you debug what's going on... it inadvertently also makes this code longer than it needs to be. Newer revisions of OpenCL are moving towards simplifying this portion.

In [ ]:

    // Allocate weird OpenCL structures
    cl_event event,event1,event2;
    cl_device_id device_id;             // compute device id 
    cl_context context;                 // compute context
    cl_command_queue commands;          // compute command queue
    cl_program program;                 // compute program
    cl_kernel kernel[3];                // compute kernel

    // Get number of OpenCL devices installed
    cl_uint dev_cnt = 0;
    clGetPlatformIDs(0, 0, &dev_cnt);
    
    // Get the list of platform identifiers
    cl_platform_id platform_ids[5];

    // Check type of device you want to target
    clGetPlatformIDs(dev_cnt, platform_ids, NULL);
    for(i=0;i<dev_cnt;i++) {
#ifdef DEVICE_GPU
        err = clGetDeviceIDs(platform_ids[i], CL_DEVICE_TYPE_GPU, 1, &device_id, NULL);
#else
        err = clGetDeviceIDs(platform_ids[i], CL_DEVICE_TYPE_CPU, 1, &device_id, NULL);
#endif
        if(err == CL_SUCCESS)
            break;
    }
    
    if (err != CL_SUCCESS) {
            if(err == CL_INVALID_PLATFORM)
                    printf("CL_INVALID_PLATFORM\n");
            if(err == CL_INVALID_DEVICE_TYPE)
                    printf("CL_INVALID_DEVICE_TYPE\n");
            if(err == CL_INVALID_VALUE)
                    printf("CL_INVALID_VALUE\n");
            if(err == CL_DEVICE_NOT_FOUND)
                    printf("CL_DEVICE_NOT_FOUND\n");
            printf("Error: Failed to clGetDeviceIDs!\n");
            return EXIT_FAILURE;
    }

    // Create a compute context 
    context = clCreateContext(0, 1, &device_id, NULL, NULL, &err);
    if (!context) {
            printf("Error: Failed clCreateContext!\n");
            return EXIT_FAILURE;
    }

     // Create a command commands
     commands = clCreateCommandQueue(context, device_id, CL_QUEUE_PROFILING_ENABLE, &err);
     if (!commands) {
            printf("Error: Failed clCreateCommandQueue!\n");
            return EXIT_FAILURE;
    }

    // Read in the source file as a string. Yuck!
    char *kern_src;
    long kern_size;
    kern_size = LoadOpenCLKernel("simple.cl", &kern_src);
    if( kern_ize < 0L ) {
        perror("File read failed");
        return EXIT_FAILURE;
    }

    // Create OpenCL object to hold the program source
    program = clCreateProgramWithSource(context, 1, (const char **) & kern_src, NULL, &err);
    if (!program) {
        printf("Error: Failed clCreateProgramWithSource!\n");
        return EXIT_FAILURE;
    }

    // Compile the OpenCL kernel you've just read
    err = clBuildProgram(program, 0, NULL, NULL, NULL, NULL);
    if (err != CL_SUCCESS) {
            size_t len;
            char buf[2048];
            printf("Error: Failed clBuildProgram!\n");
            clGetProgramBuildInfo(program, device_id, CL_PROGRAM_BUILD_LOG, sizeof(buf), buf, &len);
            printf("%s\n", buf);
            exit(1);
    }

    // Create the executable OpenCL kernel 
    kernel[0] = clCreateKernel(program, "convolve", &err);
    if (!kernel[0] || err != CL_SUCCESS) {
            printf("Error: Failed clCreateKernel!\n");
            exit(1);
    }

As an experiment -- here's the same code without error checking!

In [ ]:
    // Allocate weird OpenCL structures
    cl_event event,event1,event2;
    cl_device_id device_id;             // compute device id 
    cl_context context;                 // compute context
    cl_command_queue commands;          // compute command queue
    cl_program program;                 // compute program
    cl_kernel kernel[3];                // compute kernel

    // Get number of OpenCL devices installed
    err = clGetDeviceIDs(platform_ids[i], CL_DEVICE_TYPE_GPU, 1, &device_id, NULL);
    
    // Create a compute context 
    context = clCreateContext(0, 1, &device_id, NULL, NULL, &err);

    // Create a command commands
    commands = clCreateCommandQueue(context, device_id, CL_QUEUE_PROFILING_ENABLE, &err);

    // Read in the source file as a string. Yuck!
    char *kern_src;
    LoadOpenCLKernel("simple.cl", &kern_src);

    // Create OpenCL object to hold the program source
    program = clCreateProgramWithSource(context, 1, (const char **) & kern_src, NULL, &err);

    // Compile the OpenCL kernel you've just read
    err = clBuildProgram(program, 0, NULL, NULL, NULL, NULL);

    // Create the executable OpenCL kernel 
    kernel[0] = clCreateKernel(program, "convolve", &err);

Now that the OpenCL objects have been created, kernels compiled, devices detected and targeted, we can proceed to the real meat of the code. Here, we allocated **host** and **device** memory structures. This step is important to understand. The offload model uses the accelerator to speedup portions of the code that are bottleneck in your application. When you offload, you need to copy over the data that the accelerator needs to access. This is an artifcact of the separation of memory spaces of the accelerator and host. In future OpenCL releases, this might go away as memory spaces can be unified. Some recent NVIDIA GPUs unify the spaces already. 

The device memories are allocated using **clCreateBuffer** which is similar to **malloc** on the host. There are fancy tags that provide more information to the OpenCL compiler to optimize storage of the device arrays. 

You also need to build the list of kernel arugments using **clSetKernelArg**. This is exhausting! CUDA looks so much simpler -- but the pain is worth it, and as we mentioned earlier, this only has to be done once.

We then come to the key OpenCL function **clEnqueueNDRangeKernel**. This is used to run your kernel in parallel on the accelerator. You may have noticed the **localWorkSize** and **globalWorkSize** structures -- these are used by the compiler/runtime to auto-parallelize the kernel code across all work items. For our case the **globalWorkSize** is $W$x$H$ to reflect the resolution of the image. We process each pixel independently in each OpenCL workitem. The **localWorkSize** is an optional hint that suggests how many workitems to pack into an OpenCL **Compute Unit**. For this example, this doesn't really matter so much, but this can be used to optimize performance when data sharing is concerned.

We record OpenCL runtime using the **clGetEventProfilingInfo** API. This is useful to help us determine the performance of our code.

In [ ]:
    // Read input image and initialize host arrays
    pgm_t input_pgm;
    readPGM(&input_pgm,"donald_duck_in.pgm");
    int W = input_pgm.width;
    int H = input_pgm.height;

    printf("cl:main input image resolution:%dx%d\n", W, H);

    // Host memory for images, kernels
    float  *h_input, *h_output, *h_kernel;
    
    // Allocate host memory for matrices
    h_input = (float*)malloc(sizeof(float) * W * H);
    h_kernel = (float*)malloc(sizeof(float) * K * K);
    h_output = (float*)malloc(sizeof(float) * W * H);
    int bias = 1;
        
    // OpenCL device memory for matrices
    cl_mem d_input, d_output, d_kernel;    
    
    // create OpenCL device buffers
    d_image  = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, W*H, h_input, &err);
    d_kernel = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, K*K, h_kernel, &err);
    d_output = clCreateBuffer(context, CL_MEM_WRITE_ONLY , W*H, NULL, &err);
    
    if (!d_image || !d_filter || !d_output) {
        printf("Error: Failed clCreateBuffer!\n");
        return EXIT_FAILURE;
    }

    cl_ulong time_start, time_end;
    double total_time,itime;

    // Launch OpenCL kernel
    size_t localWorkSize[2], globalWorkSize[2];

    // set OpenCL kernel arguments
    err  = clSetKernelArg(kernel[0], 0, sizeof(cl_mem), (void *)&d_input);
    err |= clSetKernelArg(kernel[0], 1, sizeof(cl_mem), (void *)&d_kernel);
    err |= clSetKernelArg(kernel[0], 2, sizeof(cl_mem), (void *)&d_output);
    err |= clSetKernelArg(kernel[0], 3, sizeof(int), (void *)&K);
    err |= clSetKernelArg(kernel[0], 4, sizeof(cl_mem), (void*)&bias);

    if (err != CL_SUCCESS) {
        printf("Error: Failed clSetKernelArg! %d\n", err);
        return EXIT_FAILURE;
    }
    
    // local workgroup size sets pixels per compute unit.
    localWorkSize[0] = 2;
    localWorkSize[1] = 2;

    // global workgroup size sets total work
    globalWorkSize[0] = W;
    globalWorkSize[1] = H;
    
    // Enqueue task for parallel execution
    clGetEventProfilingInfo(event, CL_PROFILING_COMMAND_START, sizeof(time_start), &time_start, NULL);
    err = clEnqueueNDRangeKernel(commands, kernel[0], 2, NULL, globalWorkSize, localWorkSize, 0, NULL, &event);
    if (err != CL_SUCCESS){
        if(err == CL_INVALID_WORK_ITEM_SIZE)
            printf("CL_INVALID_WORK_ITEM_SIZE \n");
        if(err == CL_INVALID_WORK_GROUP_SIZE)
            printf("CL_INVALID_WORK_GROUP_SIZE \n");
        printf("Error: Failed to execute kernel! %d\n", err);
        return EXIT_FAILURE;
    }
    
    clWaitForEvents(1,&event);
    clGetEventProfilingInfo(event, CL_PROFILING_COMMAND_END, sizeof(time_end), &time_end, NULL);
        
    // Retrieve result from device
    err = clEnqueueReadBuffer(commands, d_output, CL_TRUE, 0, mem_size_output, h_output, 0, NULL, NULL);
    if (err != CL_SUCCESS) {
        printf("Error: Failed clEnqueueReadBuffer! %d\n", err);
        return EXIT_FAILURE;
    }

    pgm output_pgm;
    output_pgm.width = W;
    output_pgm.height = H;
    normalizeF2PGM(&output_pgm, h_output);
    writePGM(&output_pgm,"donald_duck_out.pgm");

    printf("cl:main timing %0.3f us\n", total_time / 1000.0);

    // Free memory and OpenCL objects
    destroyPGM(&input_pgm);
    destroyPGM(&output_pgm);
    free(h_input);
    free(h_output);
    clReleaseMemObject(d_input);
    clReleaseMemObject(d_output);
    clReleaseMemObject(d_kernel);
    clReleaseProgram(program);
    clReleaseKernel(kernel[0]);
    clReleaseCommandQueue(commands);
    clReleaseContext(context);
